# 讀出所有方志階層檔，並存於 Book obj 中

In [1]:
filepath = '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC'

In [2]:
import os

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(filepath):
    for file in f:
        if '._' not in file and '.xml' in file and file != 'bookId.xml':
            files.append(os.path.join(r, file))

print(len(files))
print(files[0:5])

3998
['/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/35392.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/97486.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/35386.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/92850.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/36842.xml']


In [3]:
import xmltodict, json

Books = {}

def singleChapter(INchapter, INparent, chapterTitles):
    _title = INchapter['title']
    
    _from = ""
    if '@from' in INchapter:
        _from = INchapter['@from']
    
    _to = ""
    if '@to' in INchapter:
        _to = INchapter['@to']
        
    _level = ""
    if '@level' in INchapter:
        _level = INchapter['@level']
        
    _n = ""
    if '@n' in INchapter:
        _n = INchapter['@n']

    # 'parent' = [sub]
#     print(INparent, _title)
    if INparent not in chapterTitles:
        chapterTitles[INparent] = [_title]
    elif _title not in chapterTitles[INparent]:
        chapterTitles[INparent].append(_title)
        
    # [sub] = [parent]
#     if _title not in chapterTitles:
#         chapterTitles[_title] = [INparent]
#     elif INparent not in chapterTitles[_title]:
#         chapterTitles[_title].append(INparent)
    
def section(INarray, INtitle, chapterTitles):
#     print("section::", INtitle, type(INarray), INarray)
    if type(INarray) == dict:
        singleChapter(INarray, INtitle, chapterTitles)
        if 'section' in INarray:
            section(INarray['section'], INarray['title'], chapterTitles) 
        
    elif type(INarray) == list:    
        for ch in INarray:
            section(ch, INtitle, chapterTitles)
            if 'section' in ch:
                section(ch['section'], ch['title'], chapterTitles)
                
    else:
        print(INarray)
        print("ERROR:", type(INarray))
        
    return chapterTitles

# file: [35393].xml
def recursiveSection(INdata):
    chapterTitles = {}
    book = INdata['title']

    if 'section' in INdata:
        titleObj = section(INdata['section'], book, chapterTitles)
    return titleObj

def parseFile(file):
    with open(file) as fp:
        xmlStr = fp.read()
        convertedDict = xmltodict.parse(xmlStr);
        jsonStr = json.dumps(convertedDict, ensure_ascii=False)
        jsonObj = json.loads(jsonStr)

#         print(jsonObj)
        book = jsonObj['book']

        titleObj = recursiveSection(book)
#         print(titleObj)
    
        rstart = file.rfind('/')
        rend = file.rfind('.')
        # file: [35393].xml
        bid = file[rstart+1:rend]
        
        bname = list(titleObj.keys())[0]         
        Books[bid] = {'book': bname, 'data': titleObj}
    
# print(files[0])
# parseFile(files[0])
# print(Books)
# parseFile("/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/97725.xml")

In [4]:
# root = AnyNode(id="root")
for file in files:
    parseFile(file)
print(len(Books))
# parseFile(files[0])

3998


In [5]:
# the first 
# print(Books[list(Books.keys())[0]])
# print(Books['17106'])
# print(Books['97725'])
print(Books['35282'])


{'book': '涇縣續志', 'data': {'涇縣續志': ['山澤附（闕）', '節婦（闕）', '涇縣續志序', '涇縣續志纂修姓氏', '涇縣續志目錄', '卷一', '卷二', '卷三', '卷四', '卷五', '卷六', '卷七', '卷八', '卷九'], '卷一': ['學校', '書院', '壇廟', '公館', '橋梁', '津渡', '古蹟', '寺觀'], '書院': ['學田'], '古蹟': ['冡墓'], '卷二': ['食貨', '職官表', '選舉表'], '食貨': ['蠲賑', '恩賚', '户口', '積貯'], '選舉表': ['例仕', '吏仕', '武選舉', '武職', '贈封', '歷襲'], '卷三': ['人物'], '人物': ['名臣', '宦業', '忠節', '孝友', '儒林', '文苑', '武功', '懿行', '尚義', '五世同堂', '百歳', '隱逸', '藝術', '寓賢'], '卷四': ['人物'], '卷五': ['人物'], '卷六': ['人物'], '卷七': ['列女'], '列女': ['已旌', '烈婦：已旌', '貞女', '壽婦', '節婦：已故待旌', '烈婦', '孝女', '現年節婦'], '卷八': ['列女'], '卷九': ['藝文', '災祥', '附刻煙石嶺禁碑并\uedce字']}}


# 哪幾本方志階層不用修正 

In [6]:
import sys

def showLevelGap(level):
    x = ""
    for idx in range(0, level):
        x += "==="
    return x
    
def showSubChapter(subs, data, level, uppers):
    if level > 10:
        sys.exit()

    for s in subs:
        prefix = showLevelGap(level)
        print(prefix, s)
        if s not in data:
            continue

        if level == 1:
#             print("LEVEL 1::", uppers)
            uppers = []
        else:
            if s in uppers:
                continue
            else:
                uppers.append(s)
            
        nxtSubs = data[s]
        showSubChapter(nxtSubs, data, level+1, uppers)
    
# read one gazetters
def showGazetter(bid):
    if bid not in Books:
        print("ERROR!!", bid)
        return
    gazetteer = Books[bid]
    title = gazetteer['book']
    print(title)

    data = gazetteer['data']
#     print(data)
    
    firstLevels = data[title]
    uppers = []
    showSubChapter(firstLevels, data, 1, uppers)
            
# showGazetter('35392')
# showGazetter('17106')
# showGazetter('97725')
# showGazetter('35282')


In [7]:
import sys
import re

def isChineseOnly(word):
    if type(word) != str:
        return False
            
    founds = re.findall(r'[\u4e00-\u9fff]+', word)
    if len(founds) != 1:
#         print("Something not Chinese:", word, founds)
        return False

def checkSubChapter(subs, data, level, uppers, limitLen):
    if level > 10:
        sys.exit()

    for s in subs:        
        # ##FIX##, the s could be dict in 97972. {'hi': '田產附', '#text': '義學'}
        if type(s) != str:
            return False
        
        if isChineseOnly(s) == False:
            return False
        
        if len(s) > limitLen:
            return False
        
        if s not in data:
            continue
            
        if level == 1:
            uppers = []
        else:
            if s in uppers:
                continue
            else:
                uppers.append(s)

        nxtSubs = data[s]
        if checkSubChapter(nxtSubs, data, level+1, uppers, limitLen) == False:
            return False
    return True

# check gazetter's chapter categories are correct or not
def checkGazetter(bid, limitLen):
    if bid not in Books:
        print("ERROR!!", bid)
        return False
    gazetteer = Books[bid]
    title = gazetteer['book']

    data = gazetteer['data']
    firstLevels = data[title]
    return checkSubChapter(firstLevels, data, 1, [], limitLen)

# checkGazetter('36789', 3)
# checkGazetter('98012', 3)
# checkGazetter('97725', 3)
# checkGazetter('97972', 10)
checkGazetter('97308', 10)


False

In [8]:
gazetters3 = []
for b in Books:
    if checkGazetter(b, 3):
        gazetters3.append(b)
print(len(gazetters3))
print(gazetters3)

17
['98145', '97953', '97622', '94355', '97017', '93658', '93659', '94291', '92903', '37039', '71417', '93555', '98144', '93051', '16649', '93602', '97806']


In [9]:
showGazetter('97806')

南川縣志
=== 沿革
=== 星野
=== 形勝
=== 城池
=== 公署
=== 學校
=== 風俗
=== 戶口
=== 田賦
=== 山川
=== 橋樑
=== 祠觀
=== 古蹟
=== 秩官
=== 節烈
=== 名宦
=== 鄉賢
=== 人物


## Merge the categories

In [10]:
categories = {}

def addCategory(parent, subarr):
    if type(parent) != str:
        return

    if parent not in categories:
        # remove the duplicated
        categories[parent] = list(set(subarr))
    else:
        existList = set(categories[parent])
        subList = set(subarr)
        differList = subList - existList
        categories[parent].extend(list(differList))
        
def mergeSubChapter(subs, data, level, uppers):
    if level > 10:
        sys.exit()

    for s in subs:
        if s not in data:
            if level == 1:
                addCategory(s, [])
            continue

        if level == 1:
#             print("LEVEL 1::", uppers)
            uppers = []
        else:
            if s in uppers:
                continue
            else:
                uppers.append(s)

        nxtSubs = data[s]
        addCategory(s, nxtSubs)
        mergeSubChapter(nxtSubs, data, level+1, uppers)
    
# read one gazetters
def mergeGazetter(bid):
    if bid not in Books:
        print("ERROR!!", bid)
        return
    gazetteer = Books[bid]
    title = gazetteer['book']
    print(title)

    data = gazetteer['data']
#     print(data)
    
    firstLevels = data[title]
    uppers = []
    mergeSubChapter(firstLevels, data, 1, uppers)

In [31]:
for bid in gazetters3:
    mergeGazetter(bid)
# showGazetter('97622')
# mergeGazetter('97622')
# categories

黟縣續志
潛江縣志稿
興濟縣志
梅菉賦志
徽縣志
南江縣備造新編志書清册
冕寧縣志清册
續定海縣志
昭覺縣志稿
荔波縣志稿
隨志
打拉池縣丞志
灤志補
麗江縣志
翁源縣志
合水縣志
南川縣志


In [12]:
removeList = ['序', '下卷', '上卷', '卷七', '序言', '自叙', '徽縣志']

# remove the elements which should not be the class
def removeElements(ctgrs, removes):
    newCategory = {}
    for key in ctgrs.keys():
        subArr = ctgrs[key]
        if key in removes:
            # remove the class and move the sub classes to upper layer
            for cs in subArr:
                if cs not in removeList and cs not in newCategory:
                    newCategory[cs] = []
        else:
            newCategory[key] = subArr
            
    return newCategory


categs = removeElements(categories, removeList)
# categs

{'人物志': ['列女', '封贈', '國戚', '孝行', '壽官', '尚義', '仕蹟', '聖蹟', '科第', '貞烈', '宦蹟'],
 '列女': ['貞女', '節烈', '節孝'],
 '建置': [],
 '政績錄': ['城守', '典吏', '訓導', '教諭', '縣丞'],
 '兵事錄': [],
 '耆舊錄': ['戶口', '名宦祠', '人類', '氏族', '節烈祠', '鄉賢祠', '忠孝祠'],
 '宗教': [],
 '實業': [],
 '地理': [],
 '水': [],
 '古蹟': [],
 '橋梁': [],
 '道路': [],
 '山': [],
 '坊表': [],
 '物產': [],
 '祠廟': [],
 '市鎮': [],
 '學堂': [],
 '圖例': [],
 '天文志': ['分野', '祥異'],
 '地理志': ['沿革', '疆域'],
 '山川志': [],
 '建置志': ['徭役',
  '公署',
  '田賦',
  '塩政',
  '職官',
  '坊表',
  '鎮店',
  '馬政',
  '窑厰',
  '城池',
  '祠廟',
  '戶口',
  '支應官',
  '鋪舍',
  '古蹟',
  '官地',
  '莊村',
  '寺觀',
  '屯所'],
 '風土志': ['時序', '風土', '鄉儀', '土產'],
 '學校志': ['儒學', '社學', '文廟', '學額', '義學'],
 '金石志': [],
 '證': [],
 '首': [],
 '東': ['封典古', '武古', '武今', '功名今', '舉貢古', '职官古'],
 '西': [],
 '北': [],
 '中': [],
 '風俗': [],
 '嵗例': [],
 '善舉': [],
 '災祥': ['疫', '兵事'],
 '诗': [],
 '節烈': [],
 '選舉': [],
 '公署': [],
 '土産': [],
 '教諭': [],
 '孝友': [],
 '仙釋': [],
 '寺院': [],
 '貢賦': [],
 '兵防': [],
 '州判': [],
 '典史': [],
 '疆域': [],
 '祥異': [],
 '流寓': [

# Check length > 4

In [13]:
def checkChineseOnly(INctgrs):
    for key in INctgrs.keys():
        isChineseOnly(key)
        subArr = INctgrs[key]
        
        for elm in subArr:
            isChineseOnly(elm)
            
checkChineseOnly(categs)

In [14]:
gazetters4 = []
for b in Books:
    if b not in gazetters3 and checkGazetter(b, 4):
        gazetters4.append(b)
print(len(gazetters4))
print(gazetters4)

40
['97679', '98012', '93915', '97523', '97107', '97813', '97346', '93749', '92932', '93252', '35748', '92482', '98068', '40997', '71730', '93761', '36561', '35454', '17180', '16646', '36574', '94404', '16640', '35598', '94205', '93889', '845', '346', '36416', '16696', '852', '380', '17313', '97805', '17035', '643', '836', '97648', '97810', '35792']


In [15]:
# showGazetter('97679')
for bid in gazetters4:
    mergeGazetter(bid)
# showGazetter('97622')
# mergeGazetter('97622')
# categories

玉山縣志
昭萍志略
鄰水縣續志
餘杭縣志稿
朝城縣志略
巴州志略
三續淄川縣志
朝邑縣志
鄖陽志補
富民縣志
敦煌縣志
臺拱縣文獻紀要
巫山縣志
齊乘
濱州志
西林縣志
雙山縣志
宜川縣志
商河縣志
仁化縣志
梨樹縣志
靖海衛志
臨朐縣志
趙城縣志
鳳縣志
懋功縣志
連山綏瑤廳志
始興縣志
重修儀徵縣志
壽州志
南澳志
百色廳志
豐乘
開縣志
古海陵縣志
玉門縣志
續番禺縣志
略陽縣志
巫山縣志
長陽縣志


{'人物志': ['列女',
  '封贈',
  '國戚',
  '孝行',
  '壽官',
  '尚義',
  '仕蹟',
  '聖蹟',
  '科第',
  '貞烈',
  '宦蹟',
  '列傳',
  '列女傳',
  '先賢',
  '流寓',
  '孝義',
  '文學',
  '旌異',
  '進士',
  '歲貢',
  '名賢',
  '鄉賢',
  '薦舉',
  '武勳',
  '例貢',
  '貞節',
  '舉人',
  '宦績中',
  '孝友',
  '方外',
  '忠烈',
  '宦績上',
  '宦績下',
  '義行',
  '隱逸',
  '藝術',
  '僑寓',
  '儒行'],
 '列女': ['貞女', '節烈', '節孝', '賢惠', '貞烈'],
 '建置': ['津梁',
  '邱墓',
  '古蹟',
  '陂塘',
  '行署',
  '壇廟',
  '公署',
  '坊表',
  '收卹',
  '倉庫',
  '驛舗',
  '城池'],
 '政績錄': ['城守', '典吏', '訓導', '教諭', '縣丞'],
 '兵事錄': [],
 '耆舊錄': ['戶口', '名宦祠', '人類', '氏族', '節烈祠', '鄉賢祠', '忠孝祠'],
 '宗教': [],
 '實業': [],
 '卷七': ['地理',
  '水',
  '古蹟',
  '橋梁',
  '道路',
  '山',
  '坊表',
  '物產',
  '祠廟',
  '市鎮',
  '學堂',
  '秩官',
  '武備志',
  '人物志',
  '經政畧',
  '輿地志'],
 '圖例': [],
 '天文志': ['分野', '祥異'],
 '地理志': ['沿革', '疆域'],
 '山川志': ['圍井', '古蹟', '丘墓附'],
 '建置志': ['徭役',
  '公署',
  '田賦',
  '塩政',
  '職官',
  '坊表',
  '鎮店',
  '馬政',
  '窑厰',
  '城池',
  '祠廟',
  '戶口',
  '支應官',
  '鋪舍',
  '古蹟',
  '官地',
  '莊村',
  '寺觀',
  '屯所',
  '城防',
  '文廟',
  '堡寨',
  '兵制',

In [16]:
gazettersN = []
for b in Books:
#     print(b)
    if checkGazetter(b, 30):
        gazettersN.append(b)
print(len(gazettersN))

1549


In [17]:
# showGazetter('97679')
for bid in gazettersN:
    mergeGazetter(bid)
# showGazetter('97622')
# mergeGazetter('97622')
# categories

建寧縣志
弋陽縣志
天長縣志纂輯志稿
蒙陰縣志
無錫志
茶陵縣志
靈川縣志
太康縣志
岳池縣志
續纂光澤縣志
封丘縣志
崇寧縣志
黟縣續志
新城縣志
臨縣志
信邑志稿
無爲縣小志
山東通志
岳陽縣志
德安府志
欒城縣志
潛江縣志稿
陵水縣志
江華縣志
黔陽縣志
建德縣志
太倉州志
懷仁縣志
蓬溪縣志
壽張縣志
洪雅縣續志
江寧縣志
西安縣志
東臺縣志稿
湖廣圖經志書
重修陝西乾州志
邢臺縣志
冀縣志
陝西志輯要
青神縣志
樂亭縣志
鉛山縣志
淶水縣志
建陽縣志
鷄澤縣志
石泉縣志
沁水縣志
懷安縣志
懷安縣志
唐縣志
大寧縣志
續修郟縣志
合肥縣志
寶應圖經
吉安府志
玉山縣志
懷遠縣志
惠安縣志
金山縣志
淳安縣志
會昌縣志
眉州屬志
長寧縣志
筠連縣志
確山縣志
涉縣志
海寧縣志
黔南識略
彭水縣志
寶坻縣志
樂平縣志
湘潭縣志
昭萍志略
武陵縣志
續滕縣志
重修臺灣縣志
襄城縣志
沔縣志
四川總志
臨山衛志
漢陽縣志
續修平利縣志
利津縣續志
小海場新志
翁源縣新志
臨朐續志
灃縣縣志
興文縣志
興寧縣志
平湖縣志
丹棱縣志
行唐縣新志
湖南通志
廉州府志
崇慶縣志
續修臺灣府志
績溪縣志
靜寧州志
永安縣次志
開泰縣志
續修羅江縣志
榆次縣志
安定縣志
鄰水縣志
吉安縣志
續修曲阜縣志
泗虹合志
續興安府志
續修臺灣縣志
臨沂縣志
續修藍田縣志
興濟縣志
烏青鎮志
曲沃縣志
郫縣志
順義縣志
大竹縣志
新登縣志
玉門縣志
江西全省輿圖
新化縣志
臨潼縣志
鄰水縣續志
商南縣志
鄖陽志
沙河縣志
鹽源縣志
鄒縣續志
蒲江縣志
藍山縣志
浦江志略
藍山縣圖志
房縣志鈔
平鄉縣志
新修潼川府志
汝陽縣志
新建縣志
富陽縣輿地小志
營山縣志
靈壽縣志
餘杭縣志稿
華容縣志
續補虞鄉縣新志
懷來縣志
補纂仁壽縣原志
齊東縣志
名山縣新志
梅菉賦志
直隸綿州志
故城縣志
乾州志
重輯楓涇小志
永川縣志
榆次縣志
應州再續志
瀘州直隸州志
宿州志
道州志
回疆志
浯溪新志
嵊縣志
嘉定州志
安東縣志
保安縣志略
烏青文獻
寶山縣志
榮昌縣志
安塞縣志
南和縣志
重修崑山縣志
新會縣志
豐潤縣志
江油縣志
寧鄉縣志
鹿邑縣志
朝城縣志略
長興縣志
德清縣續志
新修成都府志
簡陽縣志
東安縣志
安福縣志
江陰縣志
興寧縣志
禮縣志
鍾祥縣志
欒城縣志
南溪縣志
魚臺縣志
續修

{'人物志': ['列女',
  '封贈',
  '國戚',
  '孝行',
  '壽官',
  '尚義',
  '仕蹟',
  '聖蹟',
  '科第',
  '貞烈',
  '宦蹟',
  '列傳',
  '列女傳',
  '先賢',
  '流寓',
  '孝義',
  '文學',
  '旌異',
  '進士',
  '歲貢',
  '名賢',
  '鄉賢',
  '薦舉',
  '武勳',
  '例貢',
  '貞節',
  '舉人',
  '宦績中',
  '孝友',
  '方外',
  '忠烈',
  '宦績上',
  '宦績下',
  '義行',
  '隱逸',
  '藝術',
  '僑寓',
  '儒行',
  '善士',
  '儒林',
  '宦業',
  '文苑',
  '忠義',
  '寓賢',
  '方技',
  '忠節',
  '烈女',
  '仙釋',
  '宦績',
  '才技',
  '藝文',
  '武功',
  '勳德',
  '祭塟',
  '罷婦',
  '兵官',
  '武弁',
  '技術',
  '恩廕',
  '賢哲',
  '例據',
  '義門',
  '科貢',
  '彚進',
  '貤恩',
  '名彥',
  '奏薦',
  '旌表',
  '節孝',
  '壽民',
  '義夫',
  '行誼',
  '名臣',
  '忠臣',
  '忠勇',
  '學業',
  '先正',
  '實行',
  '仕績',
  '介節',
  '理學',
  '篤行',
  '孪子',
  '名',
  '貢士',
  '節',
  '治行',
  '耆壽',
  '儒碩',
  '勳烈',
  '學林',
  '鄉善',
  '經濟',
  '橋寓',
  '舊列女什麼',
  '善良',
  '殉難',
  '貨殖',
  '節烈',
  '新增節烈女表',
  '四先生傳',
  '壽婦',
  '貞孝',
  '貞烈表',
  '賢媛',
  '舊列女編姓表',
  '選舉二',
  '列女三',
  '節儀',
  '賢達',
  '耆老',
  '仕宦',
  '人物序',
  '德行',
  '選舉',
  '隱逆',
  '壽年',
  '孝子',
  '游寓',
  '雜記',
  '人物',
  '后妃'

In [18]:
specialWords = ['卷', '一', '二', '三', '四', '五', '六', '七', '八', '九', '十', ]

# test

In [19]:
# dupbooks = {}
# for key in bookTitles:
# #     print(bookTitles[key])
#     if len(bookTitles[key]) > 1:
# #         print(key, bookTitles[key])
#         dupbooks[key] = bookTitles[key]
# print(len(dupbooks))

In [20]:
# import pandas as pd

# resDf = pd.DataFrame.from_dict(dupbooks, orient='index')
# # print(resDf)
# resDf.sort_index(inplace=True)
# resDf.head()

# print(len(resDf))

In [21]:
# outputPath = '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志'
# write_file_to = '{0}/{1}'.format(outputPath, 'gazetteers_20190811_v2.xlsx')
# resDf.to_excel(write_file_to, index=True)

In [22]:
# import pandas as pd

# resDf = pd.DataFrame(chapterTitles, columns=['section'])
# resDf.sort_values(by=['section'], inplace=True)
# resDf.head()

# print(len(resDf))

In [23]:
# outputPath = '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志'
# write_file_to = '{0}/{1}'.format(outputPath, 'gazetteers_20190811_v1.xlsx')
# resDf.to_excel(write_file_to, index=False)

In [24]:
# count = 0
# for key in chapterTitles:
#     print(key, chapterTitles[key])
#     count += 1
#     if count > 5:
#         break

In [25]:
# # Remove bookname
# booknamecnt = 0
# for sec in chapterTitles:
# #     print(type(sec))
#     if sec == None:
#         continue
#     titles = chapterTitles[sec]
    
#     for tit in titles:
#         if tit == None:
#             continue
# #         print(sec)
#         if sec.startswith(tit):
#             if tit not in bookTitles:
#                 # remove the same section names in parent and sub section both
#                 if sec == tit:
#                     chapterTitles[sec].remove(tit)
#                     print(sec, tit)
#             else:
# #                 print(tit)
#                 chapterTitles[sec].remove(tit)

#             booknamecnt += 1
        
#         # remove parent section is book
# #         if tit in bookTitles:
# #             chapterTitles[sec].remove(tit)
# #             booknamecnt += 1
# #             print(sec, tit)
            
# print(booknamecnt)
# # print(len(chapterTitles))

In [26]:
# bookTitles['成安縣志']

In [27]:
# remove parent section is book
# for sec in chapterTitles:
# #     print(type(sec))
#     if sec == None:
#         continue
#     titles = chapterTitles[sec]
    
#     for tit in titles:
#         if tit == None:
#             continue

#         if tit in bookTitles:
#             # remove the parent section name which is the same as bookname
#             chapterTitles[sec].remove(tit)
            
# print(len(chapterTitles))

In [28]:
# 僊居水卷之一
# type(chapterTitles)

In [29]:
# import re

# for key in chapterTitles:
#     titles = chapterTitles[key]
#     for t in titles:
#         x = re.search(".*卷[首之上下一二三四五六七八九十]*", t)
#         if x != None:
#             print(t, x)
#             chapterTitles[key].remove(t)

In [30]:
# chapterTitles